In [1]:
library(statnet)
library(stats)

Loading required package: tergm

Loading required package: ergm

Loading required package: network


'network' 1.19.0 (2024-12-08), part of the Statnet Project
* 'news(package="network")' for changes since last version
* 'citation("network")' for citation information
* 'https://statnet.org' for help, support, and other information



'ergm' 4.8.1 (2025-01-20), part of the Statnet Project
* 'news(package="ergm")' for changes since last version
* 'citation("ergm")' for citation information
* 'https://statnet.org' for help, support, and other information


'ergm' 4 is a major update that introduces some backwards-incompatible
changes. Please type 'news(package="ergm")' for a list of major
changes.


Loading required package: networkDynamic


'networkDynamic' 0.11.5 (2024-11-21), part of the Statnet Project
* 'news(package="networkDynamic")' for changes since last version
* 'citation("networkDynamic")' for citation information
* 'https://statnet.org' for help, support, and other informatio

In [2]:
el_diplomatic = read.csv("../data/2011/features/dip_exhange_clean.csv", stringsAsFactors = FALSE)
diplomatic_exchange_net = network(el_diplomatic, directed = TRUE, matrix.type = "edgelist")
diplomatic_exchange_net

ERROR: Error: `loops` is `FALSE`, but `x` contains loops.
The following values are affected:
	- `x[1540, 1:2]`
	- `x[1579, 1:2]`


In [3]:
el_colony = read.csv("../data/common/colonization_el.csv", stringsAsFactors = FALSE)
colony_net = network(el_colony, directed = TRUE, matrix.type = "edgelist")
set.edge.value(colony_net, "colonization", el_colony$colonization)
colony_net

 Network attributes:
  vertices = 174 
  directed = TRUE 
  hyper = FALSE 
  loops = FALSE 
  multiple = FALSE 
  bipartite = FALSE 
  total edges= 30276 
    missing edges= 0 
    non-missing edges= 30276 

 Vertex attribute names: 
    vertex.names 

 Edge attribute names not shown 

In [4]:
el = read.csv("../data/2011/edgelist_threshold.csv", stringsAsFactors = FALSE)
nl = read.csv("../data/2011/nodelist.csv", stringsAsFactors = FALSE)

In [5]:
nl$gdp_us_dollar <- log(nl$gdp_us_dollar)
nl$area <- log(nl$area)
nl$population <- log(nl$population)
nl$gdp_per_capita <- log(nl$gdp_per_capita)

In [6]:
nl$gdp_us_dollar = as.numeric(scale(nl$gdp_us_dollar))
nl$gdp_growth = as.numeric(scale(nl$gdp_growth))
nl$inflation_rate = as.numeric(scale(nl$inflation_rate))
nl$population = as.numeric(scale(nl$population))
nl$gdp_per_capita = as.numeric(scale(nl$gdp_per_capita))
nl$agriculture_forestry_fishing_of_gdp = as.numeric(scale(nl$agriculture_forestry_fishing_of_gdp))
nl$industry_of_gdp = as.numeric(scale(nl$industry_of_gdp))
nl$merchandise_of_gdp = as.numeric(scale(nl$merchandise_of_gdp))
nl$net_barter_of_trade = as.numeric(scale(nl$net_barter_of_trade))
nl$foreign_direct_investment_inflows = as.numeric(scale(nl$foreign_direct_investment_inflows))

In [7]:
net = network(el, directed = TRUE, matrix.type = 'edgelist', vertex.attr=nl, vertex.attrnames=colnames(nl), ignore.eval = FALSE, names.eval='trade')
set.edge.value(net, "trade", el$weight)
net

 Network attributes:
  vertices = 160 
  directed = TRUE 
  hyper = FALSE 
  loops = FALSE 
  multiple = FALSE 
  bipartite = FALSE 
  total edges= 3594 
    missing edges= 0 
    non-missing edges= 3594 

 Vertex attribute names: 
    agriculture_forestry_fishing_of_gdp area continent country_iso3 foreign_direct_investment_inflows gdp_growth gdp_per_capita gdp_us_dollar industry_of_gdp inflation_rate landlocked langoff_1 merchandise_of_gdp net_barter_of_trade population vertex.names 

 Edge attribute names not shown 

In [ ]:
model = ergm(
                        net ~
                        edges +
                        mutual +
                        triangles + 
                        nodecov("gdp_us_dollar") + # world bank
                        absdiff("gdp_us_dollar") + # world bank
                        nodecov("inflation_rate") + # world bank
                        absdiff("inflation_rate") + # world bank
                        nodecov("gdp_growth") + # world bank
                        absdiff("gdp_growth") + # world bank
                        nodematch("continent") + # http://www.cepii.fr/CEPII/en/bdd_modele/download.asp?id=6
                        nodematch("landlocked") + # http://www.cepii.fr/CEPII/en/bdd_modele/download.asp?id=6
                        nodematch("langoff_1") + # http://www.cepii.fr/CEPII/en/bdd_modele/download.asp?id=6
                        edgecov(diplomatic_exchange_net) + # https://pardee.du.edu/diplomatic-representation-data-set 
                        nodecov("agriculture_forestry_fishing_of_gdp") + # world bank
                        absdiff("agriculture_forestry_fishing_of_gdp") + # world bank
                        nodecov("industry_of_gdp") + # world bank
                        absdiff("industry_of_gdp") + # world bank
                        nodecov("merchandise_of_gdp") + # world bank
                        absdiff("merchandise_of_gdp") + # world bank
                        nodecov("net_barter_of_trade") + # world bank
                        absdiff("net_barter_of_trade") + # world bank
                        nodecov("foreign_direct_investment_inflows") + # world bank
                        absdiff("foreign_direct_investment_inflows") + # world bank
                        edgecov(colony_net, "colonization")
                        , control = control.ergm(MCMLE.maxit = 40, MCMC.interval = 20000)
                )

Starting maximum pseudolikelihood estimation (MPLE):

Evaluating the predictor and response matrix.

Maximizing the pseudolikelihood.

Finished MPLE.

Starting Monte Carlo maximum likelihood estimation (MCMLE):

Iteration 1 of at most 40:



In [3]:
summary(model)

ERROR: Error: object 'model' not found


In [ ]:
gof <- gof(model ~ model + distance + dspartners + odegree + idegree + triadcensus)

In [ ]:
gof

In [ ]:
plot(gof)

In [ ]:
mcmc.diagnostics(model, which = c("plots"))